Connected to venv (Python 3.13.2)

In [1]:
import os
import django
from django.conf import settings
from django.apps import AppConfig

os.chdir('/Users/sg44574/Dropbox/Coding/nba-analytics-dashboard2')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'nba_analytics_project.settings')
django.setup()

In [2]:
"""
Django settings for nba_analytics_project project.

Generated by 'django-admin startproject' using Django 5.1.7.

For more information on this file, see
https://docs.djangoproject.com/en/5.1/topics/settings/

For the full list of settings and their values, see
https://docs.djangoproject.com/en/5.1/ref/settings/
"""

from pathlib import Path


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/5.1/howto/deployment/checklist/


import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = os.getenv('SECRET_KEY')
# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = os.getenv('DEBUG')

# Database configuration
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': os.getenv('DB_NAME'),
        'USER': os.getenv('DB_USER'),
        'PASSWORD': os.getenv('DB_PASSWORD'),
        'HOST': os.getenv('DB_HOST'),
        'PORT': os.getenv('DB_PORT'),
    }
}



ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    # Default apps
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    
    # Third-party apps
    
    # Local apps (we'll create these next)
    'nba_data',
    'dashboard',
]

MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]

ROOT_URLCONF = "nba_analytics_project.urls"

TEMPLATES = [
    {
        "BACKEND": "django.template.backends.django.DjangoTemplates",
        "DIRS": [],
        "APP_DIRS": True,
        "OPTIONS": {
            "context_processors": [
                "django.template.context_processors.debug",
                "django.template.context_processors.request",
                "django.contrib.auth.context_processors.auth",
                "django.contrib.messages.context_processors.messages",
            ],
        },
    },
]

WSGI_APPLICATION = "nba_analytics_project.wsgi.application"


# Database
# https://docs.djangoproject.com/en/5.1/ref/settings/#databases

DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.sqlite3",
    }
}


# Password validation
# https://docs.djangoproject.com/en/5.1/ref/settings/#auth-password-validators

AUTH_PASSWORD_VALIDATORS = [
    {
        "NAME": "django.contrib.auth.password_validation.UserAttributeSimilarityValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.MinimumLengthValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.CommonPasswordValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.NumericPasswordValidator",
    },
]


# Internationalization
# https://docs.djangoproject.com/en/5.1/topics/i18n/

LANGUAGE_CODE = "en-us"

TIME_ZONE = "UTC"

USE_I18N = True

USE_TZ = True


# Static files (CSS, JavaScript, Images)
# https://docs.djangoproject.com/en/5.1/howto/static-files/

STATIC_URL = "static/"

# Default primary key field type
# https://docs.djangoproject.com/en/5.1/ref/settings/#default-auto-field

DEFAULT_AUTO_FIELD = "django.db.models.BigAutoField"


In [3]:
import os
from typing import Dict, Any, List, Optional, Tuple
from balldontlie import BalldontlieAPI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class NBAApiClient:
    """Client for interacting with the balldontlie API using the official package"""
    
    def __init__(self):
        # Get API key from environment variable
        api_key = os.getenv('BALLDONTLIE_API_KEY')
        if not api_key:
            raise ValueError("BALLDONTLIE_API_KEY environment variable is required")
        
        self.api = BalldontlieAPI(api_key=api_key)
    
    def get_all_teams(self) -> List[Dict]:
        """Get all NBA teams"""
        response = self.api.nba.teams.list()
        
        # Handle ListResponse object - get the data attribute directly
        if hasattr(response, 'data'):
            # Convert NBATeam objects to dictionaries
            teams = []
            for team in response.data:
                team_dict = {
                    'id': team.id,
                    'abbreviation': team.abbreviation,
                    'city': team.city,
                    'conference': team.conference,
                    'division': team.division,
                    'full_name': team.full_name,
                    'name': team.name
                }
                teams.append(team_dict)
            return teams
        return []
    
    def get_players(self, per_page: int = 25, cursor: Optional[int] = None) -> Tuple[List[Dict], Optional[int]]:
        """
        Get paginated list of players
        
        Args:
            per_page: Number of players per page
            cursor: Cursor for pagination (new pagination style)
            
        Returns:
            Tuple containing (list of player dictionaries, next cursor)
        """
        params = {
            'per_page': per_page,
            'cursor': cursor
        }
            
        response = self.api.nba.players.list(**params)
        
        # Default return values
        players = []
        next_cursor = None
        
        # Handle ListResponse object - get the data attribute directly
        if hasattr(response, 'data'):
            # Convert NBAPlayer objects to dictionaries
            for player in response.data:
                # Extract team data if available
                team_data = None
                if hasattr(player, 'team') and player.team:
                    team_data = {
                        'id': player.team.id,
                        'abbreviation': player.team.abbreviation,
                        'city': player.team.city,
                        'conference': player.team.conference,
                        'division': player.team.division,
                        'full_name': player.team.full_name,
                        'name': player.team.name
                    }
                
                player_dict = {
                    'id': player.id,
                    'first_name': player.first_name,
                    'last_name': player.last_name,
                    'position': getattr(player, 'position', ''),
                    'height': getattr(player, 'height', None),
                    'weight_pounds': getattr(player, 'weight_pounds', None),
                    'jersey_number': getattr(player, 'jersey_number', None),
                    'college': getattr(player, 'college', None),
                    'country': getattr(player, 'country', None),
                    'draft_year': getattr(player, 'draft_year', None),
                    'draft_round': getattr(player, 'draft_round', None),
                    'draft_number': getattr(player, 'draft_number', None),
                    'team': team_data
                }
                players.append(player_dict)
        
        # Extract next cursor if available
        if hasattr(response, 'meta') and hasattr(response.meta, 'next_cursor'):
            next_cursor = response.meta.next_cursor
            
        return players, next_cursor
    
    def get_games(self, **params) -> Tuple[List[Dict], Optional[int]]:
        """
        Get games with optional filters
        
        Args:
            **params: Parameters to filter games by (per_page, cursor, seasons, team_ids, etc.)
            
        Returns:
            Tuple containing (list of game dictionaries, next cursor)
        """
        response = self.api.nba.games.list(**params)
        
        # Default return values
        games = []
        next_cursor = None
        
        # Handle ListResponse object - get the data attribute directly
        if hasattr(response, 'data'):
            # Convert NBAGame objects to dictionaries
            for game in response.data:
                # Extract home team data
                home_team = None
                if hasattr(game, 'home_team') and game.home_team:
                    home_team = {
                        'id': game.home_team.id,
                        'abbreviation': game.home_team.abbreviation,
                        'city': game.home_team.city,
                        'conference': game.home_team.conference,
                        'division': game.home_team.division,
                        'full_name': game.home_team.full_name,
                        'name': game.home_team.name
                    }
                
                # Extract visitor team data
                visitor_team = None
                if hasattr(game, 'visitor_team') and game.visitor_team:
                    visitor_team = {
                        'id': game.visitor_team.id,
                        'abbreviation': game.visitor_team.abbreviation,
                        'city': game.visitor_team.city,
                        'conference': game.visitor_team.conference,
                        'division': game.visitor_team.division,
                        'full_name': game.visitor_team.full_name,
                        'name': game.visitor_team.name
                    }
                
                game_dict = {
                    'id': game.id,
                    'date': game.date,
                    'datetime': getattr(game, 'datetime', None),
                    'home_team': home_team,
                    'visitor_team': visitor_team,
                    'home_team_score': game.home_team_score,
                    'visitor_team_score': game.visitor_team_score,
                    'season': getattr(game, 'season', None),
                    'status': game.status,
                    'period': getattr(game, 'period', None),
                    'time': getattr(game, 'time', None),
                    'postseason': getattr(game, 'postseason', False)
                }
                
                games.append(game_dict)
        
        # Extract next cursor if available
        if hasattr(response, 'meta') and hasattr(response.meta, 'next_cursor'):
            next_cursor = response.meta.next_cursor
            
        return games, next_cursor
    
    def _process_response(self, response) -> Dict:
        """Process API response into a dictionary format"""
        result = {}
        
        # Extract data from response
        if hasattr(response, 'data'):
            result['data'] = response.data
        
        # Extract meta information if available
        if hasattr(response, 'meta'):
            result['meta'] = {}
            meta = response.meta
            
            # Copy common meta fields
            for field in ['total_pages', 'current_page', 'next_page', 'per_page', 'total_count', 'next_cursor']:
                if hasattr(meta, field):
                    result['meta'][field] = getattr(meta, field)
        
        return result

# Create a singleton instance
api_client = NBAApiClient()

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Verify the API key is loaded
api_key = os.getenv('BALLDONTLIE_API_KEY')
print(f"API key loaded: {'Yes' if api_key else 'No'}")

API key loaded: Yes


In [5]:
from django.core.management.base import BaseCommand
import logging

logger = logging.getLogger(__name__)

class Command(BaseCommand):
    help = 'Import all NBA teams from the API'

    def handle(self, *args, **options):
        self.stdout.write('Importing NBA teams...')
        
        # Get teams from API
        try:
            teams_data = NBAApiClient().get_all_teams()
            
            if not teams_data:
                self.stdout.write(self.style.WARNING('No teams found in the API response.'))
                return
            
            self.stdout.write(f'Found {len(teams_data)} teams')
            
            # Process each team
            for team_data in teams_data:
                team, created = Team.objects.update_or_create(
                    team_id=team_data['id'],
                    defaults={
                        'abbreviation': team_data['abbreviation'],
                        'city': team_data['city'],
                        'conference': team_data['conference'],
                        'division': team_data['division'],
                        'full_name': team_data['full_name'],
                        'name': team_data['name'],
                    }
                )
                
                action = 'Created' if created else 'Updated'
                self.stdout.write(f'{action} team: {team.full_name}')
            
            self.stdout.write(self.style.SUCCESS('Successfully imported teams'))
            
        except Exception as e:
            self.stdout.write(self.style.ERROR(f'Error importing teams: {str(e)}'))
            logger.error(f'Error importing teams: {str(e)}', exc_info=True)

In [6]:
# Create the API client instance
api_client = NBAApiClient()

# Get teams data and store it in variable
teams_data = api_client.get_all_teams()

# Debug print
print(f"Number of teams found: {len(teams_data)}")
print("\nFirst team data:", teams_data[0] if teams_data else "No teams found")

ServerError: Bad Gateway

In [7]:
teams_data

[{'id': 1,
  'abbreviation': 'ATL',
  'city': 'Atlanta',
  'conference': 'East',
  'division': 'Southeast',
  'full_name': 'Atlanta Hawks',
  'name': 'Hawks'},
 {'id': 2,
  'abbreviation': 'BOS',
  'city': 'Boston',
  'conference': 'East',
  'division': 'Atlantic',
  'full_name': 'Boston Celtics',
  'name': 'Celtics'},
 {'id': 3,
  'abbreviation': 'BKN',
  'city': 'Brooklyn',
  'conference': 'East',
  'division': 'Atlantic',
  'full_name': 'Brooklyn Nets',
  'name': 'Nets'},
 {'id': 4,
  'abbreviation': 'CHA',
  'city': 'Charlotte',
  'conference': 'East',
  'division': 'Southeast',
  'full_name': 'Charlotte Hornets',
  'name': 'Hornets'},
 {'id': 5,
  'abbreviation': 'CHI',
  'city': 'Chicago',
  'conference': 'East',
  'division': 'Central',
  'full_name': 'Chicago Bulls',
  'name': 'Bulls'},
 {'id': 6,
  'abbreviation': 'CLE',
  'city': 'Cleveland',
  'conference': 'East',
  'division': 'Central',
  'full_name': 'Cleveland Cavaliers',
  'name': 'Cavaliers'},
 {'id': 7,
  'abbreviati

In [ ]:
import asyncio
from asgiref.sync import sync_to_async
import pandas as pd # Make sure pandas is imported
from nba_data.analytics.dataframes import get_teams_dataframe, get_players_dataframe, get_games_dataframe
# Wrap the synchronous function call
async def get_teams_df_async():
    df = await sync_to_async(get_teams_dataframe)()
    return df
teams_df = await get_teams_df_async()

async def get_players_df_async():
    df = await sync_to_async(get_players_dataframe)()
    return df
players_df = await get_players_df_async()

async def get_games_df_async():
    df = await sync_to_async(get_games_dataframe)()
    return df
games_df = await get_games_df_async()

print(teams_df.head(), players_df.head(), games_df.head())

   team_id abbreviation       city conference   division          full_name  \
0        1          ATL    Atlanta       East  Southeast      Atlanta Hawks   
1        2          BOS     Boston       East   Atlantic     Boston Celtics   
2        3          BKN   Brooklyn       East   Atlantic      Brooklyn Nets   
3        4          CHA  Charlotte       East  Southeast  Charlotte Hornets   
4        5          CHI    Chicago       East    Central      Chicago Bulls   

      name  
0    Hawks  
1  Celtics  
2     Nets  
3  Hornets  
4    Bulls      player_id first_name      last_name position  height_feet  height_inches  \
0          1       Alex        Abrines        G          6.0            6.0   
1          2     Jaylen          Adams        G          6.0            0.0   
2          3     Steven          Adams        C          6.0           11.0   
3          4        Bam        Adebayo      C-F          6.0            9.0   
4          5   DeVaughn  Akoon-Purcell      G-F     

In [14]:
import pandas as pd
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clean_player_data(df):
    """
    Clean the player DataFrame by handling missing values and standardizing field formats.
    
    Args:
        df (pandas.DataFrame): DataFrame containing player data
        
    Returns:
        pandas.DataFrame: Cleaned player DataFrame
    """
    if df.empty:
        return df
    
    # Make a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Standardize position data
    if 'position' in cleaned_df.columns:
        # Map variations to standard positions
        position_map = {
            'G': 'G', 'SG': 'G', 'PG': 'G', 'Guard': 'G',
            'F': 'F', 'SF': 'F', 'PF': 'F', 'Forward': 'F',
            'C': 'C', 'Center': 'C',
            'G-F': 'G-F', 'F-G': 'G-F', 
            'F-C': 'F-C', 'C-F': 'F-C'
        }
        
        # Apply mapping and handle missing/unknown values
        cleaned_df['position_standard'] = cleaned_df['position'].map(
            lambda x: position_map.get(x, x) if pd.notna(x) else 'Unknown'
        )
    
    # Handle missing height/weight data
    for col in ['height_feet', 'height_inches', 'height_total_inches', 'weight_pounds']:
        if col in cleaned_df.columns:
            # Replace missing values with NaN for consistency
            cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce')
    
    return cleaned_df

def clean_teams_data(df):
    """
    Clean the team  DataFrame by handling missing values and standardizing field formats.
    
    Args:
        df (pandas.DataFrame): DataFrame containing team data
        
    Returns:
        pandas.DataFrame: Cleaned team DataFrame
    """
    if df.empty:
        return df
    
    # Make a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Standardize conference data
    if 'conference' in cleaned_df.columns:
        # Map variations to standard positions
        conference_map = {
            'East': 'East', 'West': 'West',
            'East Conference': 'East', 'West Conference': 'West',
            'Eastern Conference': 'East', 'Western Conference': 'West',
        }
        
        # Apply mapping and handle missing/unknown values
        cleaned_df['conference_standard'] = cleaned_df['conference'].map(
            lambda x: conference_map.get(x, x) if pd.notna(x) else 'Unknown'
        )
    
    # Handle missing abbreviation data
    for col in ['abbreviation']:
        if col in cleaned_df.columns:
            # Replace missing values with NaN for consistency
            cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce')
    
    return cleaned_df

def clean_games_data(df):
    """
    Clean the games DataFrame by handling missing values and standardizing field formats.
    
    Args:
        df (pandas.DataFrame): DataFrame containing games data
        
    Returns:
        pandas.DataFrame: Cleaned games DataFrame
    """
    if df.empty:
        return df
    
    # Make a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # Standardize date data to format YYYY-MM-DD
    if 'date' in cleaned_df.columns:
        cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])
    
    # Handle missing datetime data
    for col in ['datetime']:
        if col in cleaned_df.columns:
            # Replace missing values with NaN for consistency
            cleaned_df[col] = pd.to_datetime(cleaned_df[col], errors='coerce')
    
    return cleaned_df

def enhance_game_data(df):
    """
    Prepare the game DataFrame by adding derived metrics.
    
    Args:
        df (pandas.DataFrame): DataFrame containing game data
        
    Returns:
        pandas.DataFrame: Enhanced game DataFrame with derived metrics
    """
    logger.info("Initial game DataFrame shape: %s", df.shape)
    
    if df.empty:
        return df
    
    # Make a copy to avoid modifying the original
    enhanced_df = df.copy()
    
    # Calculate point differential
    if all(col in enhanced_df.columns for col in ['home_team_score', 'visitor_team_score']):
        enhanced_df['point_diff'] = enhanced_df['home_team_score'] - enhanced_df['visitor_team_score']
        enhanced_df['total_points'] = enhanced_df['home_team_score'] + enhanced_df['visitor_team_score']
    
    # Add win/loss indicator for home team
    if 'point_diff' in enhanced_df.columns:
        enhanced_df['home_team_won'] = (enhanced_df['point_diff'] > 0).astype(int)
    
    # Extract day of week, month, year, and hour
    if 'date' in enhanced_df.columns and pd.api.types.is_datetime64_any_dtype(enhanced_df['date']):
        enhanced_df['day_of_week'] = enhanced_df['date'].dt.day_name()
        enhanced_df['month'] = enhanced_df['date'].dt.month_name()
        enhanced_df['year'] = enhanced_df['date'].dt.year
        enhanced_df['hour'] = enhanced_df['datetime'].dt.hour
    
    logger.info("Enhanced game DataFrame shape: %s", enhanced_df.shape)
    logger.info("Enhanced game DataFrame summary:\n%s", enhanced_df.describe())

    
    return enhanced_df

def prepare_home_vs_away(games_df):
    """
    Calculate aggregate team statistics from game data.
    
    Args:
        df (pandas.DataFrame): DataFrame containing game data
        
    Returns:
        pandas.DataFrame: Team statistics DataFrame
    """
    if games_df.empty:
        return pd.DataFrame()
    
    # Make sure the input DataFrame has the necessary columns
    required_columns = [
        'home_team_id', 'home_team_name', 'visitor_team_id', 
        'visitor_team_name', 'home_team_score', 'visitor_team_score', 
        'season'
    ]
    
    if not all(col in games_df.columns for col in required_columns):
        missing = [col for col in required_columns if col not in games_df.columns]
        raise ValueError(f"Missing required columns: {missing}")
    
    # Create home team records
    home_team_stats = games_df.rename(columns={
        'home_team_id': 'team_id',
        'home_team_name': 'team_name',
        'home_team_score': 'points_scored',
        'visitor_team_score': 'points_allowed',
    }).assign(
        is_home=True
    )
    
    # Create visitor team records
    visitor_team_stats = games_df.rename(columns={
        'visitor_team_id': 'team_id',
        'visitor_team_name': 'team_name',
        'visitor_team_score': 'points_scored',
        'home_team_score': 'points_allowed',
    }).assign(
        is_home=False
    )
    
    # Select common columns for both home and visitor records
    columns_to_keep = [
        'team_id', 'team_name', 'season', 'date', 
        'points_scored', 'points_allowed', 'is_home'
    ]
    
    # Ensure all columns exist in the dataframes
    home_columns = [col for col in columns_to_keep if col in home_team_stats.columns]
    visitor_columns = [col for col in columns_to_keep if col in visitor_team_stats.columns]
    
    # Combine home and visitor records
    combined_stats = pd.concat([
        home_team_stats[home_columns],
        visitor_team_stats[visitor_columns]
    ], ignore_index=True)
    
    # Calculate wins and point differential
    combined_stats['won'] = combined_stats['points_scored'] > combined_stats['points_allowed']
    combined_stats['point_diff'] = combined_stats['points_scored'] - combined_stats['points_allowed']
    
    # Group by team and season to calculate aggregated stats
    team_season_stats = combined_stats.groupby(['team_id', 'team_name', 'season']).agg(
        games_played=('team_id', 'count'),
        wins=('won', 'sum'),
        points_scored_total=('points_scored', 'sum'),
        points_allowed_total=('points_allowed', 'sum'),
        point_diff_total=('point_diff', 'sum'),
        point_diff_avg=('point_diff', 'mean'),
        home_games=('is_home', 'sum'),
        home_wins=('won', lambda x: (combined_stats.loc[x.index, 'is_home'] & 
                                    combined_stats.loc[x.index, 'won']).sum())
    ).reset_index()
    
    # Calculate additional metrics
    team_season_stats['losses'] = team_season_stats['games_played'] - team_season_stats['wins']
    team_season_stats['win_pct'] = team_season_stats['wins'] / team_season_stats['games_played']
    team_season_stats['points_per_game'] = team_season_stats['points_scored_total'] / team_season_stats['games_played']
    team_season_stats['points_allowed_per_game'] = team_season_stats['points_allowed_total'] / team_season_stats['games_played']
    
    # Calculate away games and wins
    team_season_stats['away_games'] = team_season_stats['games_played'] - team_season_stats['home_games']
    team_season_stats['away_wins'] = team_season_stats['wins'] - team_season_stats['home_wins']
    
    # Calculate home and away win percentages
    home_mask = team_season_stats['home_games'] > 0
    away_mask = team_season_stats['away_games'] > 0
    
    team_season_stats.loc[home_mask, 'home_win_pct'] = (
        team_season_stats.loc[home_mask, 'home_wins'] / team_season_stats.loc[home_mask, 'home_games']
    )
    
    team_season_stats.loc[away_mask, 'away_win_pct'] = (
        team_season_stats.loc[away_mask, 'away_wins'] / team_season_stats.loc[away_mask, 'away_games']
    )
    
    return team_season_stats

In [37]:
# Step 2: Clean and transform data
cleaned_players_df = clean_player_data(players_df)
cleaned_teams_df = clean_teams_data(teams_df)
cleaned_games_df = clean_games_data(games_df)

# Step 3: Calculate team statistics

prepared_games_df = enhance_game_data(cleaned_games_df)     
team_stats_df = prepare_home_vs_away(prepared_games_df)

INFO:__main__:Initial game DataFrame shape: (1217, 16)
INFO:__main__:Enhanced game DataFrame shape: (1217, 23)
INFO:__main__:Enhanced game DataFrame summary:
            game_id                           date datetime  season  \
count  1.217000e+03                           1217        0  1217.0   
mean   2.347865e+06  2024-01-26 09:23:13.262119936      NaT  2023.0   
min    1.037593e+06            2023-10-24 00:00:00      NaT  2023.0   
25%    1.037975e+06            2023-12-08 00:00:00      NaT  2023.0   
50%    1.038305e+06            2024-01-24 00:00:00      NaT  2023.0   
75%    1.038638e+06            2024-03-15 00:00:00      NaT  2023.0   
max    1.590507e+07            2024-06-17 00:00:00      NaT  2023.0   
std    3.901796e+06                            NaN      NaN     0.0   

            period  home_team_score  visitor_team_score  home_team_id  \
count  1217.000000      1217.000000         1217.000000   1217.000000   
mean      4.054232       114.500411          112.423993 

ValueError: Missing required columns: ['home_team_abbreviation', 'visitor_team_abbreviation']